# Analyze Hacker News Trends

## The project details:

Hacker News is a popular website run by Y Combinator. It’s widely known by people in the tech industry as a community site for sharing news, showing off projects, asking questions, among other things.

In this project, you will be working with a table named hacker_news that contains stories from Hacker News since its launch in 2007. It has the following columns:

title: the title of the story
user: the user who submitted the story
score: the score of the story
timestamp: the time of the story
url: the link of the story
This data was kindly made publicly available under the MIT license.

In [ ]:
-- 1 Start by getting a feel for the hacker_news table!
-- Let’s find the most popular Hacker News stories:
SELECT * FROM hacker_news LIMIT 5;

-- Result

Query Results
title	user	score	timestamp	url
Penny Arcade – Surface Pro 3 update	vxNsr	517	2014-06-16T18:38:48Z	http://penny-arcade.com/news/post/2014/06/16/surface-pro-3-update
Hacking The Status Game	amirkhella	309	2011-04-18T19:10:06Z	http://blog.amirkhella.com/2011/04/18/the-status-game/
Postgres CLI with autocompletion and syntax highlighting	dmmalam	304	2015-07-23T16:32:39Z	https://github.com/dbcli/pgcli
Stephen Fry hits out at ‘infantile’ culture of trigger words and safe spaces	metafunctor	282	2016-04-12T07:28:16Z	http://attitude.co.uk/stephen-fry-hits-out-at-infantile-culture-of-trigger-words-and-safe-spaces/
Reversal: Australian Govt picks ODF doc standard over Microsoft	renai_lemay	191	2013-05-29T06:37:40Z	http://delimiter.com.au/2013/05/29/reversal-australian-govt-picks-odf-doc-standard/



In [ ]:
-- 2 Recent studies have found that online forums tend to be dominated by a small percentage of their users (1-9-90 Rule).
-- Is this true of Hacker News?
-- Is a small percentage of Hacker News submitters taking the majority of the points?
-- First, find the total score of all the stories.
SELECT SUM(score) from hacker_news;

-- Result

Query Results
SUM(score)
6366

In [ ]:
-- 3 Next, we need to pinpoint the users who have accumulated a lot of points across their stories.
-- Find the individual users who have gotten combined scores of more than 200, and their combined scores.
SELECT user, SUM(score) FROM hacker_news GROUP BY user HAVING SUM(score) > 200 ORDER BY 2 DESC;

-- Result

Query Results
user	SUM(score)
vxNsr	517
amirkhella	309
dmmalam	304
metafunctor	282


In [ ]:
-- 4 Then, we want to add these users’ scores together and divide by the total to get the percentage.
-- Add their scores together and divide it by the total sum. Like so:
-- SELECT (1.0 + 2.0 + 3.0) / 6.0;
-- So, is Hacker News dominated by these users?
SELECT (517 + 309 + 304 + 282) / 6366.0;

-- Result

Query Results
(517 + 309 + 304 + 282) / 6366.0
0.221803330191643

In [ ]:
-- 5 Oh no! While we are looking at the power users, some users are rickrolling — tricking readers into clicking on a link to a funny video and claiming that it links to information about coding.
-- The url of the video is:
-- https://www.youtube.com/watch?v=dQw4w9WgXcQ
-- How many times has each offending user posted this link?
SELECT user, COUNT(*) FROM hacker_news WHERE url LIKE '%watch?v=dQw4w9WgXcQ%' GROUP BY user ORDER BY 2 DESC;

-- Result

Query Results
user	COUNT(*)
sonnynomnom	2
scorpiosister	1


In [ ]:
-- 6 Hacker News stories are essentially links that take users to other websites.
-- Which of these sites feed Hacker News the most:
-- GitHub, Medium, or New York Times?
-- First, we want to categorize each story based on their source.

-- 7 Build on the previous query:
-- Add a column for the number of stories from each URL using COUNT().
-- Also, GROUP BY the CASE statement.
-- Remember that you can refer to a column in GROUP BY using a number.
SELECT CASE
  WHEN url LIKE '%github.com%' THEN 'GitHub'
  WHEN url LIKE '%medium.com%' THEN 'Medium'
  WHEN url LIKE '%nytimes.com%' THEN 'New York Times'
  ELSE 'Other'
  END AS 'Source',
  COUNT(*)
FROM hacker_news GROUP BY 1 ORDER BY 2 DESC;

-- Result

Query Results
Source	COUNT(*)
Other	3952
GitHub	23
New York Times	13
Medium	12


In [ ]:
-- 8 Every submitter wants their story to get a high score so that the story makes it to the front page, but…
-- What’s the best time of the day to post a story on Hacker News?
-- Before we get started, let’s run this query and take a look at the timestamp column:
SELECT timestamp FROM hacker_news LIMIT 10;

-- Result

Query Results
timestamp
2014-01-27T17:31:13Z
2011-10-23T18:46:40Z
2016-02-28T06:26:56Z
2014-08-12T22:13:10Z
2013-03-06T12:28:02Z
2011-04-16T21:04:23Z
2014-03-18T21:44:46Z
2012-11-19T11:54:38Z
2016-11-04T13:55:30Z
2016-07-02T22:54:47Z

In [ ]:
-- 9 SQLite comes with a strftime() function - a very powerful function that allows you to return a formatted date.
-- It takes two arguments:
-- strftime(format, column)
-- Let’s test this function out:
SELECT timestamp, strftime('%H', timestamp) FROM hacker_news GROUP BY 1
LIMIT 20;

-- Result

Query Results
timestamp	strftime('%H', timestamp)
2007-03-16T20:52:19Z	20
2007-04-03T03:04:09Z	03
2007-05-01T03:11:17Z	03
2007-05-05T05:43:58Z	05
2007-05-11T05:48:53Z	05
2007-05-25T22:07:18Z	22
2007-06-08T08:44:50Z	08
2007-07-01T00:06:57Z	00
2007-07-27T16:47:00Z	16
2007-08-08T00:45:36Z	00
2007-08-12T07:35:25Z	07
2007-08-16T01:02:49Z	01
2007-08-24T18:17:32Z	18
2007-08-27T17:32:10Z	17
2007-09-11T16:27:49Z	16
2007-09-20T20:31:21Z	20
2007-09-22T08:21:17Z	08
2007-09-26T05:03:17Z	05
2007-10-08T21:28:06Z	21


In [ ]:
-- 10 Okay, now we understand how strftime() works. Let’s write a query that returns three columns:
-- The hours of the timestamp
-- The average score for each hour
-- The count of stories for each hour
SELECT strftime('%H', timestamp), AVG(score), COUNT(*) FROM hacker_news GROUP BY 1 ORDER BY 2 DESC;

-- Result

Query Results
strftime('%H', timestamp)	AVG(score)	COUNT(*)
18	27.0277777777778	266
07	21.3333333333333	104
19	20.4137931034483	238
20	18.2758620689655	239
12	14.3103448275862	123
09	11.125	119
16	9.75555555555556	237
15	9.49122807017544	268
17	8.86046511627907	215
23	8.43478260869565	170
11	8.32258064516129	113
06	7.5	110
03	6.91304347826087	134
13	5.67647058823529	164
00	5.09090909090909	137
01	4.40740740740741	146
04	3.875	116
05	3.55172413793103	125
14	3.51515151515152	213
21	3.25	234
22	3.20833333333333	185
10	3.07894736842105	112
02	2.45	136
08	2.04166666666667	90
6


In [ ]:
-- 11 Let’s edit a few things in the previous query:
-- Round the average scores (ROUND()).
-- Rename the columns to make it more readable (AS).
-- Add a WHERE clause to filter out the NULL values in timestamp.
-- Take a look at the result again:
-- What are the best hours to post a story on Hacker News?
SELECT strftime('%H', timestamp) AS 'Hour', ROUND(AVG(score)) AS 'Average Score', COUNT(*) AS 'Number of Stories' FROM hacker_news WHERE timestamp NOT NULL GROUP BY 1 ORDER BY 2 DESC;

-- Result

Query Results
Hour	Average Score	Number of Stories
18	27.0	266
07	21.0	104
19	20.0	238
20	18.0	239
12	14.0	123
09	11.0	119
16	10.0	237
17	9.0	215
15	9.0	268
23	8.0	170
11	8.0	113
06	8.0	110
03	7.0	134
13	6.0	164
00	5.0	137
14	4.0	213
05	4.0	125
04	4.0	116
01	4.0	146
22	3.0	185
21	3.0	234
10	3.0	112
08	2.0	90
02	2.0	136

## Review
I really like this small database since it gives the opportunity to work with time. Most of the practical Business Analysis projects include time, be it for checking the difference of sales or users engaging with the organisation. All of them need to be within a certain timeframe. About the database, although it is okay with processing it in SQL, I think it would be better if it was processed with python where we can use NLP libraries and we could get more information. For example categorising the titles. There we can see which category has the most articles, which category has the most scores, which category has a stable release. After making a diagram with the stable release, we can check it against others so we can infer some kind of new trend (example: more cybersecurity articles due to a new virus)